In [8]:
import numpy as np
import pandas as pd
from collections import Counter

In [9]:
train = pd.read_json('../file/train.json', encoding='utf-8')
song_meta = pd.read_json('../file/song_meta.json', encoding='utf-8')

In [10]:
train

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000
...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000


In [12]:
songs = [__ for _ in train.songs for __ in _]

In [14]:
len(set(songs))

615142

# 노래가 100번 초과인 곡들만 선택


In [15]:
song_selected = [_[0] for _ in list(filter(lambda e: e[1] > 100, Counter(songs).items()))]


In [16]:
n = len(song_selected)
print(n)

8491


In [17]:
train["song_selected"] = train.songs.apply(lambda e: [_ for _ in e if _ in song_selected])

In [18]:
train_w_selected_song = train[train.song_selected.apply(lambda e: bool(e))]
print(len(train_w_selected_song)) # 플레이리스트중 100번 초과 곡이 list에 들어있는 플레이리스트 수

99266


## 정연님이 구현하신 embedding (훨씬 빠르고 간단, sklearn의 OneHotEncoder 사용)

In [19]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(np.array(song_selected).reshape(-1, 1))
enc.transform(np.array([497066, 389529]).reshape(-1, 1)).toarray().sum(axis=0)

array([0., 0., 0., ..., 0., 0., 0.])

In [20]:
def one_hot(train):
    X = np.array([enc.transform(np.array(song_list).reshape(-1,1)).toarray().sum(axis=0) for song_list in train.song_selected])
    return X

In [21]:
X = one_hot(train_w_selected_song)

In [22]:
x_train = X[:int(X.shape[0]*0.8)]
x_test = X[int(X.shape[0]*0.8):]
print(x_train.shape, x_test.shape)

(79412, 8491) (19854, 8491)


In [41]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import backend as K

# configure
encoding_dim = 150
input_song = Input(shape=(n,))

# layers
encoded = Dense(encoding_dim, activation='relu')(input_song)
decoded = Dense(n, activation='sigmoid')(encoded)

# Models
autoencoder = Model(input_song, decoded) # autoencoder
encoder = Model(input_song, encoded) # encoder
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input)) # decoder

print(autoencoder.layers[-2])
print(encoder.layers[-1])

In [35]:
# .from keras.datasets import mnist
from keras.layers import Input, Dense
from keras.models import Model
from keras import backend as K

# configure
encoding_dim = 150
input_song = Input(shape=(n,))

# layers
encoded = Dense(encoding_dim, activation='relu')(input_song)
decoded = Dense(n, activation='sigmoid')(encoded)

# Models
autoencoder = Model(input_song, decoded) # autoencoder
encoder = Model(input_song, encoded) # encoder
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input)) # decoder

# result viewer
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def recall(y_true, y_pred):
    y_true_yn = K.round(K.clip(y_true, 0, 1))
    y_pred_yn = K.round(K.clip(y_pred, 0, 1))

    count_true_positive = K.sum(y_true_yn * y_pred_yn)
    count_true_positive_false_negative = K.sum(y_true_yn)
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())
    return recall

# train autoencoder
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=[rmse, recall])
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=16,
                shuffle=True,
                validation_data=(x_test, x_test))

# encoding result
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

# classification train data
reducted_x_train = encoder.predict(x_train)
reducted_x_test = encoder.predict(x_test)

Train on 79412 samples, validate on 19854 samples
Epoch 1/50
 7776/79412 [=>............................] - ETA: 42s - loss: 0.6838 - rmse: 0.4953 - recall: 0.1655

KeyboardInterrupt: 

In [30]:
predicted = decoded_imgs > 0.5
accuracy = 0; precision = 0
for _ in list(zip(x_test, predicted)):
    
    TP = 0 ; TN = 0; FP = 0; FN = 0
    for __ in list(zip(_[0], _[1])):
        if __[0] == 1 and  __[1] == 1:
            TP += 1
        if __[0] == 0 and __[1] == 0:
            TN += 1
        if __[0] == 0 and __[1] == 1:
            FP += 1
        if __[0] == 1 and __[0] == 0:
            FN += 1 
    accuracy += (TP + TN) / (TP + TN + FP + FN)
    precision += TP / (TP + FP + 0.00001)
    
print('accuracy :{} recall :{}'.format(accuracy/len(x_test), precision/len(x_test)))

KeyboardInterrupt: 

In [28]:
decoded_imgs.shape

(19854, 8491)

## 6000개 training set 결과

In [51]:
from keras.datasets import mnist
from keras.layers import Input, Dense
from keras.models import Model
from keras import backend as K

# configure
encoding_dim = 32
input_img = Input(shape=(n,))

# layers
encoded = Dense(encoding_dim, activation='relu')(input_img)
decoded = Dense(n, activation='sigmoid')(encoded)

# Models
autoencoder = Model(input_img, decoded) # autoencoder
encoder = Model(input_img, encoded) # encoder
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input)) # decoder

# result viewer
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def recall(y_true, y_pred):
    y_true_yn = K.round(K.clip(y_true, 0, 1))
    y_pred_yn = K.round(K.clip(y_pred, 0, 1))

    count_true_positive = K.sum(y_true_yn * y_pred_yn)
    count_true_positive_false_negative = K.sum(y_true_yn)
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())
    return recall

# train autoencoder
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=[rmse, recall])
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=16,
                shuffle=True,
                validation_data=(x_test, x_test))

# encoding result
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

# classification train data
reducted_x_train = encoder.predict(x_train)
reducted_x_test = encoder.predict(x_test)

Train on 6792 samples, validate on 92474 samples
Epoch 1/50
6792/6792 [==============================] - 15s 2ms/step - loss: 0.6867 - rmse: 0.4968 - recall: 0.0962 - val_loss: 0.6796 - val_rmse: 0.4932 - val_recall: 0.0077
Epoch 2/50
6792/6792 [==============================] - 15s 2ms/step - loss: 0.6519 - rmse: 0.4788 - recall: 0.0047 - val_loss: 0.5724 - val_rmse: 0.4359 - val_recall: 2.6617e-07
Epoch 3/50
6792/6792 [==============================] - 15s 2ms/step - loss: 0.3138 - rmse: 0.2633 - recall: 0.0000e+00 - val_loss: 0.1110 - val_rmse: 0.1109 - val_recall: 0.0000e+00
Epoch 4/50
6792/6792 [==============================] - 17s 2ms/step - loss: 0.0639 - rmse: 0.0730 - recall: 0.0000e+00 - val_loss: 0.0403 - val_rmse: 0.0553 - val_recall: 0.0000e+00
Epoch 5/50
6792/6792 [==============================] - 15s 2ms/step - loss: 0.0326 - rmse: 0.0503 - recall: 0.0000e+00 - val_loss: 0.0278 - val_rmse: 0.0475 - val_recall: 0.0000e+00
Epoch 6/50
6792/6792 [==========================

In [54]:
predicted = reducted_x_test > 0.5
accuracy = 0; precision = 0
for _ in list(zip(x_test, predicted)):
    
    TP = 0 ; TN = 0; FP = 0; FN = 0
    for __ in list(zip(_[0], _[1])):
        if __[0] == 1 and  __[1] == 1:
            TP += 1
        if __[0] == 0 and __[1] == 0:
            TN += 1
        if __[0] == 0 and __[1] == 1:
            FP += 1
        if __[0] == 1 and __[0] == 0:
            FN += 1 
    accuracy += (TP + TN) / (TP + TN + FP + FN)
    precision += TP / (TP + FP)
    
print('accuracy :{} recall :{}'.format(accuracy/len(x_test), precision/len(x_test)))

accuracy :0.05438047712800866 recall :0.002953596569696424


## =============================================
###           최초에 짠 코드 폐기 예정
## =============================================

In [26]:
X = np.zeros(n)
for _ in train_1.song_selected_idx:
    temp = np.zeros(n)
    temp[_] = 1
    X = np.vstack((X, temp))

X = X[1:]